In [ ]:
%pip install pygis

In [1]:
import geemap
import ee


In [13]:
import ee
import geemap.core as geemap
# Construct a collection of corresponding Dynamic World and Sentinel-2 for
# inspection. Filter the DW and S2 collections by region and date.
START = ee.Date('2021-04-02')
END = START.advance(1, 'day')

col_filter = ee.Filter.And(
    ee.Filter.bounds(ee.Geometry.Point(20.6729, 52.4305)),
    ee.Filter.date(START, END),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))

# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

# Display the Dynamic World visualization with the source Sentinel-2 image.
m = geemap.Map()
m.set_center(20.6729, 52.4305, 12)
m.add_layer(
    s2_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
m

Map(center=[52.4305, 20.6729], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [1]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
from ipyleaflet import WidgetControl

# Initialize Earth Engine API
ee.Initialize()

# Load the Esri Land Cover 2020 dataset
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")
iraq = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM0_NAME', 'Iraq'))

# Define a dictionary for the legend
legend_dict = {
    "names": [
        "Water", "Trees", "Grass", "Flooded Vegetation", "Crops",
        "Scrub/Shrub", "Built Area", "Bare Ground", "Snow/Ice", "Clouds"
    ],
    "colors": [
        "#1A5BAB", "#358221", "#A7D282", "#87D19E", "#FFDB5C", "#EECFA8",
        "#ED022A", "#EDE9E4", "#F2FAFF", "#C8C8C8"
    ]
}

# Add dataset to the map
LULC_10m = esri_lulc10.mosaic()
iraq_LULC_2020 = LULC_10m.clip(iraq)

# Calculate area covered by each land cover class (sq.km)
area_image = ee.Image.pixelArea().divide(1000*1000).addBands(iraq_LULC_2020)
areas = area_image.reduceRegion(
    reducer=ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class',
    }),
    geometry=iraq.geometry(),
    scale=500,
    maxPixels=1e10
)

# Print area information
print('Area of Land Use Land Classes (in sq.km)', areas.getInfo())

# Create a legend panel manually with colors
legend_items = []
for name, color in zip(legend_dict["names"], legend_dict["colors"]):
    legend_item = widgets.HTML(value=f'<div style="display:flex; align-items:center;"><div style="width:20px; height:20px; background-color:{color}; margin-right:5px;"></div><div>{name}</div></div>')
    legend_items.append(legend_item)

legend_panel = widgets.VBox(legend_items)
legend_control = WidgetControl(widget=legend_panel, position='bottomright')

# Create a map
map1 = geemap.Map(center=[33.223191, 43.679291], zoom=7)

# Add layers to the map
map1.addLayer(iraq_LULC_2020, {'min': 1, 'max': 10, 'palette': legend_dict['colors']}, 'Iraq_LULC_2020 10m')

# Add legend control to the map
map1.add_control(legend_control)

# Display the map
display(map1)


ConnectionError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000012756725480>: Failed to resolve 'oauth2.googleapis.com' ([Errno 11001] getaddrinfo failed)"))

In [2]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
from ipyleaflet import WidgetControl

# Initialize Earth Engine API
ee.Initialize()

# Load the Esri Land Cover 2020 dataset
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")
syria = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM0_NAME', 'Syrian Arab Republic'))

# Define a dictionary for the legend
legend_dict = {
    "names": [
        "Water", "Trees", "Grass", "Flooded Vegetation", "Crops",
        "Scrub/Shrub", "Built Area", "Bare Ground", "Snow/Ice", "Clouds"
    ],
    "colors": [
        "#1A5BAB", "#358221", "#A7D282", "#87D19E", "#FFDB5C", "#EECFA8",
        "#ED022A", "#EDE9E4", "#F2FAFF", "#C8C8C8"
    ]
}

# Add dataset to the map
LULC_10m = esri_lulc10.mosaic()
syria_LULC_2020 = LULC_10m.clip(syria)

# Calculate area covered by each land cover class (sq.km)
area_image = ee.Image.pixelArea().divide(1000*1000).addBands(syria_LULC_2020)
areas = area_image.reduceRegion(
    reducer=ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class',
    }),
    geometry=syria.geometry(),
    scale=500,
    maxPixels=1e10
)

# Print area information
print('Area of Land Use Land Classes (in sq.km)', areas.getInfo())

# Create a legend panel manually with colors
legend_items = []
for name, color in zip(legend_dict["names"], legend_dict["colors"]):
    legend_item = widgets.HTML(value=f'<div style="display:flex; align-items:center;"><div style="width:20px; height:20px; background-color:{color}; margin-right:5px;"></div><div>{name}</div></div>')
    legend_items.append(legend_item)

legend_panel = widgets.VBox(legend_items)
legend_control = WidgetControl(widget=legend_panel, position='bottomright')

# Create a map
map1 = geemap.Map(center=[35.0000, 38.0000], zoom=7)  # Centered around Syria

# Add layers to the map
map1.addLayer(syria_LULC_2020, {'min': 1, 'max': 10, 'palette': legend_dict['colors']}, 'Syria_LULC_2020 10m')

# Add legend control to the map
map1.add_control(legend_control)

# Display the map
display(map1)


Area of Land Use Land Classes (in sq.km) {'groups': [{'class': 1, 'sum': 1399.0496274472982}, {'class': 2, 'sum': 1303.9877334848584}, {'class': 3, 'sum': 36.291626453124984}, {'class': 4, 'sum': 55.57855969767154}, {'class': 5, 'sum': 33407.54915144907}, {'class': 6, 'sum': 119423.1624662734}, {'class': 7, 'sum': 6585.071651024614}, {'class': 8, 'sum': 25926.486120556685}, {'class': 9, 'sum': 0.203318234375}]}


Map(center=[35.0, 38.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [60]:
dataset_id = "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"


In [61]:
esri_landcover = ee.ImageCollection(dataset_id)


In [62]:
landcover_vis = {}  # Use default visualization


In [65]:
Map = geemap.Map()
Map.addLayer(esri_landcover, landcover_vis, "ESRI Land Cover (10m)")
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
s2=ee.imagecollection(